# Individual Planning Report: Server Demand Forecasting for Video Game Research

**Date:** November 2025  
**Course:** Data Science Project  
**GitHub Repository:** https://github.com/ByronLi0207/DSI

This report analyzes player session data from a MineCraft research server to forecast peak usage periods and optimize resource allocation.

## Introduction

A research group at UBC operates a MineCraft server (https://plaicraft.ai) for collecting gameplay data. To ensure adequate server resources and software licenses, they need to predict when the highest number of simultaneous players will occur. This analysis addresses their demand forecasting needs by identifying high-traffic time windows.

In [ ]:
# Load required libraries
library(tidyverse)  # Data wrangling and visualization
library(readxl)     # Reading Excel files
library(lubridate)  # Date and time handling

# Set display options for better readability
options(repr.plot.width = 12, repr.plot.height = 8)
options(digits = 2)

---

## 1. Data Description

This section loads and describes the datasets used for demand forecasting analysis.

### 1.1 Data Loading and Initial Conversion

The data consists of two files:
- **players.xlsx**: Player profiles and characteristics
- **sessions (2).xlsx**: Individual play sessions with timestamps

For reproducibility and easier processing, we first convert these to CSV format.

In [ ]:
# Read the Excel files
players_raw <- read_excel('players.xlsx')
sessions_raw <- read_excel('sessions (2).xlsx')

# Convert to CSV for reproducible analysis
write_csv(players_raw, 'players.csv')
write_csv(sessions_raw, 'sessions.csv')

cat("✓ Data files converted to CSV format\n")
cat("  - players.csv created\n")
cat("  - sessions.csv created\n")

### 1.2 Dataset Overview

In [ ]:
# Load the CSV files for analysis
players_data <- read_csv('players.csv')
sessions_data <- read_csv('sessions.csv')

cat("=== PLAYERS DATASET ===", "\n")
cat("Dimensions:", nrow(players_data), "observations ×", ncol(players_data), "variables\n\n")

cat("Variable Summary:\n")
str(players_data)

cat("\n=== SESSIONS DATASET ===", "\n")
cat("Dimensions:", nrow(sessions_data), "observations ×", ncol(sessions_data), "variables\n\n")

cat("Variable Summary:\n")
str(sessions_data)